In [1]:
import string
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [3]:
def post_processing(pred):

    ## only use the last 3 sentences
    #proc_pred=[]
    #for p in pred:
    #    splits=[s for s in p.lower().split('\n') if s != '']
    #    proc_pred.append(' '.join(splits[-3:]).translate(str.maketrans('', '', string.punctuation)))
    
    proc_pred=[]
    for p in pred:
        splits=[s for s in p.lower().split('\n') if s != '']
        proc_pred.append(splits[-1].translate(str.maketrans('', '', string.punctuation)))
    
    new_pred=[]
    for p in proc_pred:       
        if 'response' in p:
            try: new_pred.append([t for t in p.lower().split('response')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'output' in p:
            try: new_pred.append([t for t in p.lower().split('output')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'return' in p:
            try: new_pred.append([t for t in p.lower().split('return')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'result' in p:
            try: new_pred.append([t for t in p.lower().split('result')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)                
        else:
            try: new_pred.append(p.lower().split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [4]:
ds = load_dataset("beanham/spatial_union_dataset")
test=ds['test']
gt=np.array(test['label'])
configs = [
    'zero_shot_with_heur_value_angle_cot',
    'zero_shot_with_heur_value_area_cot',
    'zero_shot_with_heur_value_angle_area_cot',    
    'few_shot_with_heur_value_angle_cot',
    'few_shot_with_heur_value_area_cot',
    'few_shot_with_heur_value_angle_area_cot'
]

In [14]:
results=[]
models=['llama3', 'mistral', '4o_mini', 'qwen_plus', '4o']

configs = [
    'zero_shot_with_heur_value_angle_cot',
    #'zero_shot_with_heur_value_area_cot',
    #'zero_shot_with_heur_value_angle_area_cot',    
    #'few_shot_with_heur_value_angle_cot',
    #'few_shot_with_heur_value_area_cot',
    #'few_shot_with_heur_value_angle_area_cot'
]
models=['4o_mini']
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)        
        metrics=metric_calculation(pred, gt)
        results.append([config, model, round(metrics[0],3), metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: 4o_mini...


In [18]:
pred=np.load(f'base/{model}/{model}_{config}.npy')
a=post_processing(pred)
wrong_index=np.where(a==2)[0]
len(wrong_index)

98

In [23]:
print(pred[wrong_index[4]])

To determine whether the two sidewalks represent the same sidewalk, we need to evaluate the parallelism condition based on the provided min_angle statistic.

### Step 1: Extract Coordinates
We have the following coordinates for the sidewalks and the road:

- **Sidewalk 1 (Sidewalk)**:
  - Coordinates: 
    - Point 1: [-122.15352219999998, 47.6177819]
    - Point 2: [-122.1535463, 47.617803399999985]
    - Point 3: [-122.1535526, 47.61833550000001]
    - Point 4: [-122.1535372, 47.61836149999999]
    - Point 5: [-122.15353780000001, 47.61844260000001]

- **Sidewalk 2 (Road)**:
  - Coordinates:
    - Point 1: [-122.1535517, 47.6182195]
    - Point 2: [-122.1535448, 47.6180968]

### Step 2: Calculate the Orientation of Each Line
To determine the orientation of each line, we can calculate the angle of the line segments formed by the coordinates.

#### Sidewalk Orientation Calculation
1. **Calculate the angle of the sidewalk**:
   - Using the first two points of the sidewalk:
     - Point A

In [15]:
results[results['model']==model]

,config,model,acc,f1
0,zero_shot_with_heur_value_angle_cot,4o_mini,0.591,0.380754
